In [1]:
import pandas as pd
train_data = pd.read_csv('./used_car_train_20200313.csv',sep=' ')
# print(train_data.head())
test_data = pd.read_csv('./used_car_testB_20200421.csv',sep=' ')
# print(test_data.head())

# print(test_data.columns)
print(test_data.notRepairedDamage.value_counts())
test_data['notRepairedDamage'].replace('-','0.0',inplace=True)#元数据中有非数值类型值，需要进行替换处理
test_data['notRepairedDamage'] = test_data.notRepairedDamage.astype('float64')#将数值类型改成float64，方便计算
train_data['notRepairedDamage'].replace('-','0.0',inplace=True)
train_data['notRepairedDamage'] = train_data.notRepairedDamage.astype('float64')

0.0    37224
-       8069
1.0     4707
Name: notRepairedDamage, dtype: int64


In [2]:
#所有字段的缺失值利用中位数补全
test_data = test_data.fillna(test_data.median())
train_data = train_data.fillna(train_data.median())

In [3]:
train_data['power'][train_data['power']>600] = 600  #对于某些字段的不在合理范围内的值进行处理
print(train_data.power.describe())

count    150000.000000
mean        116.860973
std          70.075256
min           0.000000
25%          75.000000
50%         110.000000
75%         150.000000
max         600.000000
Name: power, dtype: float64


D:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
#特征选择
features = train_data.columns.tolist()
features.remove('price')#去掉label字段
features.remove('SaleID')#去掉无关字段

In [5]:
#特征归一化
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
x = min_max_scaler.fit_transform(train_data[features].values)#全量特征值
x_test = min_max_scaler.transform(test_data[features].values)

In [6]:
#得到y值
y = train_data.price.values#全量labels
# print(y)

#数据集切分
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2)

In [7]:
#搭建模型
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(300,activation='relu',input_shape=[len(features)]),#全连接mlp
    keras.layers.Dense(200,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(1)
])


#对模型设置编辑器
model.compile(loss='mean_absolute_error',optimizer='Adam')
#模型训练 全量数据集
model.fit(x,y,batch_size=1024,epochs=200)


Epoch 1/200
147/147 [==============================] - 1s 4ms/step - loss: 4887.0977
Epoch 2/200
147/147 [==============================] - 1s 4ms/step - loss: 2767.6482
Epoch 3/200
147/147 [==============================] - 1s 5ms/step - loss: 1284.7671
Epoch 4/200
147/147 [==============================] - 1s 4ms/step - loss: 1082.7249
Epoch 5/200
147/147 [==============================] - 1s 5ms/step - loss: 992.2139
Epoch 6/200
147/147 [==============================] - 0s 3ms/step - loss: 931.6008
Epoch 7/200
147/147 [==============================] - 1s 4ms/step - loss: 884.1552
Epoch 8/200
147/147 [==============================] - 1s 5ms/step - loss: 845.6981
Epoch 9/200
147/147 [==============================] - 1s 4ms/step - loss: 809.2331
Epoch 10/200
147/147 [==============================] - 1s 4ms/step - loss: 778.3328
Epoch 11/200
147/147 [==============================] - 1s 5ms/step - loss: 748.9449
Epoch 12/200
147/147 [==============================] - 1s 4ms/step - 

147/147 [==============================] - 1s 5ms/step - loss: 480.9529
Epoch 193/200
147/147 [==============================] - 0s 3ms/step - loss: 480.8716
Epoch 194/200
147/147 [==============================] - 1s 4ms/step - loss: 483.4491
Epoch 195/200
147/147 [==============================] - 1s 5ms/step - loss: 483.4967
Epoch 196/200
147/147 [==============================] - 1s 5ms/step - loss: 479.2321
Epoch 197/200
147/147 [==============================] - 1s 5ms/step - loss: 479.9333
Epoch 198/200
147/147 [==============================] - 1s 5ms/step - loss: 480.2012
Epoch 199/200
147/147 [==============================] - 1s 5ms/step - loss: 481.1651
Epoch 200/200
147/147 [==============================] - 1s 5ms/step - loss: 478.3631


In [8]:

#模型评估
from sklearn.metrics import mean_absolute_error
print('训练集评估：MAE:',mean_absolute_error(train_y,model.predict(train_x)))

print('验证集评估：MAE:',mean_absolute_error(test_y,model.predict(test_x)))

训练集评估：MAE: 488.5341827835719
验证集评估：MAE: 485.86083263727824


In [14]:
#输出结果 调整结果数据  输出文件 提交天池     最终得分532.7958
predict_y = model.predict(x_test)
result = pd.DataFrame()
result['SaleID'] = test_data['SaleID']
result['price'] = predict_y
result['price'][result.price<11] = 11
result['price'][result.price>99999] = 99999
result.to_csv('./mlp_nn.csv',index=False)

D:\python\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
result['price'][result.price>99999]

Series([], Name: price, dtype: float32)